# **Chains in LangChain**

## **Outline**

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers.

In [85]:
! pip install langchain langchain_groq

### **LLMChain**

For using `ChatGroq`, we first need to set API Key.

In [86]:
import os

os.environ["GROQ_API_KEY"] = "YOUR-API-KEY"

In [87]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [88]:
llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.9
)

In [89]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Create the chain:
chain = LLMChain(llm = llm,
                 prompt = prompt)

In [90]:
product = "Hats with shape of animation characters."

# Invoke the chain
chain.run(product)

'What a fun business idea! Here are some name suggestions for a company that makes hats with shapes of animation characters:\n\n1. **ToonTop Hats**: A playful combination of "toon" (short for cartoon) and "top hats" that immediately conveys the idea of animation-themed hats.\n2. **Character Caps**: This name highlights the colorful characters featured on the hats, and "caps" is a colloquial term for hats.\n3. **Animate Headwear**: This name leverages the word "animate" to create a connection with animation characters, and "headwear" is a broad term that encompasses hats.\n4. **Fable Fibers**: "Fable" evokes the idea of fictional stories and characters, while "fibers" references the materials used to make the hats.\n5. **Cartoon Crowns**: This name incorporates "cartoon" to tie in with animation characters, and "crowns" is a regal term that suggests a fun, playful vibe.\n6. **Pixel Perfect Hats**: "Pixel" nods to the digital world of animation, while "perfect hats" implies high-quality 

## **SimpleSequentialChain**

- The simplest form of a sequential chain is where each step has a single input and output.

- The output of one step is passed as input to the next step in the chain. You would use `SimpleSequentialChain` it when you have a linear pipeline where each step has a single input and output. `SimpleSequentialChain` implicitly passes the output of one step as input to the next.

- This is great for composing a precise sequence of LLMChains where each builds directly on the previous output.

In [91]:
from langchain.chains import SimpleSequentialChain

In [92]:
llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.8
)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Give just one name."
)

chain_one = LLMChain(
    llm = llm,
    prompt = first_prompt
)

In [93]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following, make sure to use the company name in this description. \
    company:{company_name}"
)

# chain 2
chain_two = LLMChain(
    llm = llm,
    prompt = second_prompt
)

In [94]:
overall_simple_chain = SimpleSequentialChain(
    chains = [chain_one, chain_two],
    verbose = True
)

overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
What a fun question!

Here's a suggestion:

**ToonCaps**

This name plays off the idea of "toons" (short for cartoons or animation) and "caps" (short for hats or caps). It's catchy, easy to remember, and immediately conveys the idea that the company makes hats with animated characters on them.
Here is a 20-word description for ToonCaps:

"ToonCaps brings imagination to life with vibrant, animated character hats, empowering self-expression and playfulness in kids and kids-at-heart alike worldwide."

> Finished chain.


'Here is a 20-word description for ToonCaps:\n\n"ToonCaps brings imagination to life with vibrant, animated character hats, empowering self-expression and playfulness in kids and kids-at-heart alike worldwide."'

## **SequentialChain**

- A more general form of sequential chain allows multiple inputs and outputs per step.

- You would use `SequentialChain` when you have a more complex pipeline where steps might have multiple inputs and outputs.

- `SequentialChain` allows you to explicitly specify all the input and output variables at each step and map outputs from one step to inputs of the next. This provides more flexibility when steps might have multiple dependencies or produce multiple results to pass along.

In [95]:
from langchain.chains import SequentialChain

In [96]:
llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.9
)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

# First chain
chain_one = LLMChain(
    llm = llm,
    prompt = first_prompt,
    output_key = "English_Review"
)

In [97]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# chain 2: input= English_Review and output= summary
chain_two = LLMChain(
    llm = llm,
    prompt = second_prompt,
    output_key = "summary"
)

In [98]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review and output= language
chain_three = LLMChain(
    llm = llm,
    prompt = third_prompt,
    output_key = "language"
)

In [99]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm = llm,
                      prompt = fourth_prompt,
                      output_key = "followup_message"
                     )

In [100]:
# overall_chain: input = Review
# and output = English_Review, summary, language, followup_message
overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["Review"],
    output_variables = ["English_Review", "summary", "language", "followup_message"],
    verbose = True
)

In [101]:
overall_chain("I think that is a wonderful product, It's price is a little high but I think It worths it.")



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "I think that is a wonderful product, It's price is a little high but I think It worths it.",
 'English_Review': 'Here is the review translated to English:\n\n"I think this is a wonderful product. Although the price is a bit steep, I believe it\'s worth it."',
 'summary': 'Here is a summary of the review in 1 sentence:\n\nThe reviewer highly recommends the product, considering it worth the steep price.',
 'language': 'The language of the review is likely non-native English, possibly from an Asian language such as Chinese or Japanese. The grammatical errors and sentence structure suggest that the writer is not a native English speaker.\n\nSpecifically, the errors include:\n\n* "I think that is a wonderful product" should be "I think it\'s a wonderful product" or "I think that it\'s a wonderful product"\n* "It\'s price is a little high" should be "The price is a little high" or "Its price is a little high"\n* "I think It worths it" should be "I think it\'s worth it"\n\nDespite

## **Router Chain**

- Router chains allow routing inputs to different destination chains based on the input text. This allows the building of chatbots and assistants that can handle diverse requests.

  - Router chains examine the input text and route it to the appropriate
  destination chain.
  - Destination chains handle the actual execution based on the input.
  - Router chains are powerful for building multi-purpose chatbots/assistants.


In [102]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [103]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [104]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [105]:
llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.05
)

In [106]:
destination_chains = {}
for p_info in prompt_infos:

  # Extract the name and prompt template
  name = p_info["name"]
  prompt_template = p_info["prompt_template"]

  prompt = ChatPromptTemplate.from_template(template = prompt_template)
  chain = LLMChain(llm = llm, prompt = prompt)
  destination_chains[name] = chain

print(destination_chains)

{'physics': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"))]), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x797118799cc0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7971187a77f0>, model_name='llama3-70b-8192', temperature=0.05, groq_api_key=SecretStr('**********'))), 'math': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into

In [107]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

destinations_str = "\n".join(destinations)

In [108]:
print(destinations)
print("----------------\n")
print(destinations_str)

['physics: Good for answering questions about physics', 'math: Good for answering math questions', 'History: Good for answering history questions', 'computer science: Good for answering computer science questions']
----------------

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [109]:
default_prompt = ChatPromptTemplate.from_template("{input}")

default_chain = LLMChain(
    llm = llm,
    prompt = default_prompt)

In [110]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [111]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinations_str
)

router_prompt = PromptTemplate(
    template = router_template,
    input_variables = ["input"],
    output_parser = RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [112]:
router_template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nphysics: Good for answering questions about physics\nmath: Good for answering math q

In [113]:
chain = MultiPromptChain(
    router_chain = router_chain,
    destination_chains = destination_chains,
    default_chain = default_chain,
    verbose = True
    )

In [114]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is blackbody radiation?'}
> Finished chain.


"Blackbody radiation is a fundamental concept in physics, and I'm happy to explain it in simple terms.\n\nBlackbody radiation is the thermal radiation emitted by an idealized perfect absorber of electromagnetic radiation, known as a blackbody. In other words, it's the radiation emitted by an object when it's heated.\n\nHere's the key idea: when an object is heated, its atoms or molecules gain energy and start vibrating rapidly. As they vibrate, they emit photons across a wide range of frequencies, including visible light, infrared, and even X-rays. This emitted radiation is what we call blackbody radiation.\n\nThe fascinating thing about blackbody radiation is that its spectrum (the distribution of energy across different frequencies) is universal, meaning it doesn't depend on the material properties of the object. This is known as Planck's law, named after Max Planck, who first described it in the early 20th century.\n\nIn essence, blackbody radiation is a fundamental process that hel

In [115]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is the value of 2 + 2'}
> Finished chain.


"A classic!\n\nTo answer this question, I'll break it down into its simplest component parts:\n\n* 2: a single value\n* 2: another single value\n* +: the operation of addition\n\nNow, let's evaluate each component part:\n\n* 2: the value is simply 2\n* 2: the value is simply 2\n* +: the operation of addition means we need to combine the two values\n\nNow, let's put the component parts together:\n\n2 + 2 = ?\n\nTo answer this, I'll perform the operation of addition:\n\n2 + 2 = 4\n\nTherefore, the value of 2 + 2 is 4."

In [116]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is the biological significance of DNA in human cells?'}
> Finished chain.


"I'm a physics professor, not a biologist! While I can appreciate the importance of DNA, I must admit that I'm not the best person to answer this question. DNA is a molecule that contains the genetic instructions used in the development and function of all living organisms, but the specifics of its biological significance in human cells are outside my area of expertise.\n\nHowever, I can suggest that you consult with a biologist or a biochemist who can provide a more detailed and accurate answer. They can explain how DNA is involved in the transmission of genetic information, the synthesis of proteins, and the regulation of gene expression, among other things.\n\nAs a physicist, I'll stick to what I know best: the fascinating world of atoms, particles, and forces!"